# L4 – Word embedding

### Resources
1. Possible [truncated svd](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) implementation of sklearn.
2. Pytorch [tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) for embeddings.
3. GloVe [site](https://nlp.stanford.edu/projects/glove/).
4. FastText [site](https://fasttext.cc).
5. [Gensim](https://radimrehurek.com/gensim/) library.

**Word embedding** – the collective name for a set of language modeling and feature learning techniques in natural language processing where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with much higher dimensionality. Word and phrase embeddings, when used as the underlying input representation, have been shown to boost the performance in NLP tasks such as syntactic parsing, sentiment analysis and translation. Usually, problem reduces to unsupervised learning on a large corpus of text. So, learning process is based on the idea that context of word closely associated with it. For example, the context can be a document in which the word is located or adjacent words.

### 1. Data

As text corpus you may use simple english [wikipedia](https://simple.wikipedia.org/wiki/Simple_English_Wikipedia).
1. 131K articles
2. 2K common English words.

Also you can use just english [wikipedia](https://en.wikipedia.org/wiki/Wikipedia):
1. Includes 5М+ articles
2. 2M+ different tokens

#### Exercises
1. Download all wikipedia articles.
2. Make all preprocessing work that you need (e.g. remove markup).

In [ ]:
import nltk
import os
import re
import sys
import glob
import nltk
import gensim
import numpy as np
import scipy
import pandas as pd
from tqdm import tqdm
from uuid import uuid4
from functools import reduce
from multiprocessing import Pool
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install wikiextractor

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!python -m wikiextractor.WikiExtractor -o ../data/wiki/ --processes 8 '/content/gdrive/My Drive/simplewiki-20210120-pages-articles-multistream.xml.bz2'

INFO: Preprocessing '/content/gdrive/My Drive/simplewiki-20210120-pages-articles-multistream.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Loaded 27463 templates in 53.8s
INFO: Starting page extraction from /content/gdrive/My Drive/simplewiki-20210120-pages-articles-multistream.xml.bz2.
INFO: Using 8 extract processes.
INFO: Extracted 100000 articles (487.2 art/s)
INFO: Extracted 200000 articles (594.5 art/s)
INFO: Finished 8-process extraction of 247406 articles in 434.5s (569.4 art/s)


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def _remove_non_printed_chars(string):
    reg = re.compile('[^a-zA-Zа-яА-ЯёЁ]')
    return reg.sub(' ', string)

def _remove_stop_words(string,sw=[]):
    return ' '.join([word if word not in sw else '' \
                     for word in string.strip().split(' ')])

def _trim_string(string):
    return re.sub('\s+',' ',string).strip().lower()

def clean_string(string,
                 stop_words_list,
                 min_len=2,
                 max_len=30):

    string = _remove_non_printed_chars(string)
    string = _remove_stop_words(string,stop_words_list)
    string = _trim_string(string)
  
    string = ' '.join(gensim.utils.simple_preprocess(string,
                                                     min_len=min_len,
                                                     max_len=max_len))
    return string

def splitkeepsep(s, sep):
    cleaned = []
    s = re.split("(%s)" % re.escape(sep), s)
    for _ in s:
        if _!='' and _!=sep:
            cleaned.append(sep+_)
    return cleaned

def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_special_chars(text,char_list):
    for char in char_list:
        text=text.replace(char,'')
    return text.replace(u'\xa0', u' ')   

def process_wiki_files(wiki_file):
    chars = ['\n']
    global sw
    
    with open(wiki_file, encoding='utf-8') as f:
        content = f.read()

    articles = splitkeepsep(content,'<doc id=')
    df = pd.DataFrame(columns=['article_uuid', 'words after'])
    
    for article in articles:
        uuid = uuid4()
        
        article = remove_special_chars(remove_html_tags(article),
                                       chars)        
        
        sentences = nltk.sent_tokenize(article)
        proc_sentences = [clean_string(sentence,sw) for sentence in sentences]
        words = [sentence.split() for sentence in proc_sentences]
        words = [item for sublist in words for item in sublist]
        #words = np.unique(np.array(words).flatten())
      
        temp_df = pd.DataFrame(
            {'article_uuid': [uuid]*len(words),
             'words after': words
            })
        df = df.append(temp_df)
    
    return df

def list_multiprocessing(param_lst,
                         func,
                         **kwargs):
    
    workers = kwargs.pop('workers')

    with Pool(workers) as p:
        apply_lst = [([params], func, i, kwargs) for i,params in enumerate(param_lst)]
        result = list(tqdm(p.imap(_apply_lst, apply_lst), total=len(apply_lst)))

    result=sorted(result,key=lambda x:x[0])
    return [_[1] for _ in result]

def _apply_lst(args):
    params, func, num, kwargs = args
    return num, func(*params,**kwargs)

wiki_files = []

for filename in glob.iglob('data/wiki/*/*', recursive=True):
    wiki_files.append(filename)
    
# plain list of stop words
sw_en = set(stopwords.words('english'))
sw_ru = set(stopwords.words('russian'))
sw = list(sw_ru.union(sw_en))  

df = list_multiprocessing(wiki_files,
                          process_wiki_files,
                          workers=4)

df = pd.concat(df).reset_index(drop=True)
df.article_uuid = df.article_uuid.astype(str)
df.to_csv('data/ruwiki.csv')

100%|██████████| 150/150 [10:55<00:00,  4.37s/it]


In [ ]:
df

,article_uuid,words after
0,db0b1903-7ca5-4302-91a4-2cc787fd34d7,study
1,db0b1903-7ca5-4302-91a4-2cc787fd34d7,electrical
2,db0b1903-7ca5-4302-91a4-2cc787fd34d7,properties
3,db0b1903-7ca5-4302-91a4-2cc787fd34d7,biological
4,db0b1903-7ca5-4302-91a4-2cc787fd34d7,cells
...,...,...
13912106,b97108d8-1579-4696-a2d3-f2efb0202799,united
13912107,b97108d8-1579-4696-a2d3-f2efb0202799,states
13912108,b97108d8-1579-4696-a2d3-f2efb0202799,as
13912109,b97108d8-1579-4696-a2d3-f2efb0202799,census


### 2. LSA (Latent semantic analysis)
This solution uses full document as a context of word. So, we have some vocabulary $W$ and a set of documents $D$. Matrix $X$ with shape $|W| \times |D|$ at position $w, d$ stores importance of word $w$ for document $d$. If word $w$ is not found in the document $d$ than at appropriate position $X$ has 0 (obviously, matrix is sparse).

For each matrix you can find [SVD decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
$$X = U \Sigma V^{T} \text{, где }$$
* $U$ – orthogonal matrix $|W| \times |W|$ of left singular vectors
* $\Sigma$ – diagonal matrix $|W| \times |D|$ of singular values
* $V$ – orthogonal matrix $|D| \times |D|$  of right singular vectors

Let's suppouse that row $w$ in matrix $U\Sigma$ is a vector that represents word $w$, and row $d$ of $V$ coresponds to document $d$. In some sense we already found the embeddings of words and documents at the same time. But size of vectors are determined by documents number $|D|$.

Nevertheless you can use truncated SVD instead
$$ X \approx X_k = U_k \Sigma_k V^{T}_k \text{, where }$$
* $U_k$ – $k$ left singular vectors
* $\Sigma_k$ – diagonal matrix $|k| \times |k|$ of largest singular values
* $V_k$ – $k$ right singular vectors

Also it known that $X_k$ is the best approximation of $X$ in the term of [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm) for all matrices of rank $k$.

So, it is possible to obtain a compressed words' representations of size $k \ll |W|$ as rows of matrix $U_k \Sigma_k $. It just remains to understand how to calculate the original values of the matrix $X$. There is a set of approaches, here are some of them
1. Binary flag that document contains the word.
2. The number occurrences of word in document.
3. Word frequency that is better known as $tf$ (it's possible also use $\ln(1 + tf)$).
4. [$tf \cdot idf$](https://ru.wikipedia.org/wiki/TF-IDF).

#### Further readning
1. You also can read this [aticle](https://en.wikipedia.org/wiki/Latent_semantic_analysis).
2. By the way, some [modification](http://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) of SVD decompostion won at [Netflix prize](https://en.wikipedia.org/wiki/Netflix_Prize).
3. It is easy to see that the problem of SVD decomposition is reduced to finding eigenvectors of matrix $X \cdot X^T$.
4. You can find [here](https://arxiv.org/abs/0909.4061) probabilistic algorithms of matrix decomposition.

#### Exercises
1. Find matrix $X$, using you favourite approach (maybe you need the sparse matrix class of scipy library).
2. Find word embeddings of size $k = 128$.
3. Implement k-nearest neighbor search for Euclidean norm.
4. Find 10 nearest words for the set of words: 'cat', 'loves', 'clever' and 'mandarin'.
5. Repeat experiment for cosinus as metric. What are the results?

In [ ]:
data_gr = df.groupby(by=['article_uuid'], as_index=False).agg(lambda x: x.tolist())

In [ ]:
data_gr

,article_uuid,words after
0,00005930-8dff-449d-935d-d6e44f41f74b,"[kgalagadi, districtkgalagadi, district, south..."
1,000092b4-0a66-4aaf-a35a-06fb72aa1a29,"[heartlandheartland, american, western, drama,..."
2,0000f9f1-a666-4ad0-9e49-5b59938fafab,"[osamu, shimomura, japanese, organic, chemist,..."
3,0001180f-ad22-4681-8275-707be4ea7546,"[list, districts, west, bengalthe, himalayas, ..."
4,00013b3c-35cc-43b3-ac99-e8116b01cc3f,"[agnetha, ltskogagnetha, se, ltskog, born, apr..."
...,...,...
245910,ffff878d-3e90-4d0f-ae67-980e21fc606c,"[jos, ignacio, zah, nosjos, ignacio, zah, nos,..."
245911,ffff95a4-8f1f-47c5-bb87-d7f4789138d6,[pedigree]
245912,ffffb45a-b838-4d24-a146-9eb084997113,"[lightspeed, magazine, lightspeed, american, o..."
245913,ffffb90f-b6cb-4419-b3f4-14e538545e80,"[alan, turing]"


In [ ]:
data = data_gr['words after'].values.tolist()

for i in range(len(data)):
    data[i] = ' '.join(map(str, data[i]))

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

In [ ]:
X = X.T

In [ ]:
X.shape

(447685, 245915)

In [ ]:
U, S, W = scipy.sparse.linalg.svds(X, k=128)

In [ ]:
emb = U.dot(np.diag(S))

In [ ]:
emb

array([[-1.87848030e-04, -2.67323820e-03,  3.82352630e-03, ...,
        -6.02750627e-03,  5.66510118e-03,  1.71542949e-02],
       [ 3.10386913e-03, -2.45129145e-03, -2.37916415e-03, ...,
         1.01770205e-03,  1.00109995e-02,  1.24872869e-02],
       [-4.52574843e-05,  8.03229013e-05,  3.22703399e-04, ...,
        -3.52892269e-04,  3.14634035e-04,  8.71731655e-04],
       ...,
       [-4.88941032e-05, -8.25596084e-05,  1.28389641e-05, ...,
        -2.92427033e-05,  6.72782899e-05,  1.56582609e-04],
       [ 2.92541634e-05, -1.39058148e-05, -1.46193479e-04, ...,
        -1.00658798e-04, -4.31114500e-05,  3.79053198e-04],
       [-4.55827648e-05, -2.14146912e-05, -6.98265032e-05, ...,
        -3.67912121e-05,  3.96820897e-05,  1.22956176e-04]])

In [ ]:
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return np.sqrt(distance)

In [ ]:
def cos_distance(row1, row2):
    return scipy.spatial.distance.cosine(row1, row2)

In [ ]:
def get_neighbors(train, word, num_neighbors, norm):
    test_row = train[word]
    distances = list()
    for el in train:
        train_row = train[el]
        dist = norm(test_row, train_row)
        distances.append((el, dist))      
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

In [ ]:
cos_distance(emb[77], emb[2])

-0.04912724279344194

In [ ]:
d = dict(zip(vectorizer.get_feature_names(), emb))

In [ ]:
get_neighbors(d, 'cat', 10, euclidean_distance)

['cat',
 'cats',
 'dog',
 'description',
 'feathers',
 'adults',
 'bones',
 'breed',
 'shark',
 'eating']

In [ ]:
get_neighbors(d, 'cat', 10, cos_distance)

['cat',
 'cats',
 'siamese',
 'dog',
 'adaptations',
 'creature',
 'paws',
 'catthe',
 'creatures',
 'colorful']

In [ ]:
get_neighbors(d, 'loves', 10, euclidean_distance)

['loves',
 'funny',
 'ugly',
 'humor',
 'orphan',
 'marries',
 'inspiration',
 'lovers',
 'dinner',
 'drunk']

In [ ]:
get_neighbors(d, 'loves', 10, cos_distance)

['loves',
 'friends',
 'friend',
 'happy',
 'witch',
 'funny',
 'angel',
 'magic',
 'marries',
 'humor']

In [ ]:
get_neighbors(d, 'clever', 10, euclidean_distance)

['clever',
 'inherit',
 'fashioned',
 'realize',
 'shocked',
 'jealousy',
 'persuade',
 'tricked',
 'pleased',
 'disguise']

In [ ]:
get_neighbors(d, 'clever', 10, cos_distance)

['clever',
 'kindness',
 'telling',
 'really',
 'knew',
 'good',
 'asks',
 'thinks',
 'realize',
 'thought']

In [ ]:
get_neighbors(d, 'mandarin', 10, euclidean_distance)

['mandarin',
 'simplified',
 'cantonese',
 'linguistic',
 'pronunciation',
 'linguists',
 'hakka',
 'loanwords',
 'romanization',
 'bantu']

In [ ]:
get_neighbors(d, 'mandarin', 10, cos_distance)

['mandarin',
 'cantonese',
 'subvarieties',
 'yue',
 'chinesemandarin',
 'guoyu',
 'huayu',
 'likebecame',
 'likethus',
 'menr']

In [ ]:
get_neighbors(d, 'diana', 10, cos_distance)

['diana',
 'emily',
 'joan',
 'grace',
 'julia',
 'olivia',
 'audrey',
 'nicole',
 'ingrid',
 'patricia']

### 3. word2vec
Let's consider perhaps the most popular model of word embeddings. This is mainly due to the fact that the received vectors have interesting properties. In particular, semantically similar words have close vectors, moreover linear operation of subtraction has meaning in the resulting vector space! The context of word at this time is a window of size $2c+1$, where interested word is located in the middle.

#### Continuous bag-of-words
The core idea is very simple. There is some very long text
$$w_1, w_2, \dots, w_T.$$
Consider some word $w_t$ and its context in radius $c$, namely words
$$w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}.$$
Intuition tells us that the context often really well defines the word in the middle. Then let our model restore the central word if its context is known
$$\mathcal{L} = - \frac{1}{T}\sum_{t} \log \mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}].$$

For each word $w_t$ there is some vector $v_t$, but if word $w_{t+i}$ is some context than we use vector $v'_{t+i}$. In this case we can represent context of word as the following sum
$$s'_t = \sum_{-c \leq i \leq c, i \neq 0} v'_{t+i}$$

And define probabilty with softmax
$$\mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t-1}, \dots w_{t+c}] = \frac{ \exp {s'}_t^T \cdot v_t}{\sum_j \exp {s'}_t^T \cdot v_j}.$$

Note that this model is easy to present as 2-layer neural network:
* The input is a sparse vector of dimension $|W|$ which at position $w'$ has one if word $w'$ is in context.
* Further, we have matrix $V'$ of vectors $v'$.
* Further, without any non-linearity matrix $V$ of vector $v$.
* And then the softmax layer.

#### Skip-gram
Another model predicts context for some word. The design is approximately the same, we need to optimize
$$\mathcal{L} = -\frac{1}{T} \sum_t \sum_{-c \leq i \leq c, i \neq 0} \log \mathbb{P}[w_{t+i}|w_t].$$

The probability is approximated by the following model
$$\mathbb{P}[w_{t+i}|w_t] = \frac{ \exp {v'}_{t+i}^T \cdot v_t}{\sum_j \exp {v'}_{j}^T \cdot v_t}.$$


#### Hierarchical softmax
Creation of this mode was guided to reduce the complexity of learning. However, computing of sofmax's denominator is really expensive operation. Therefore, in practice, a few other models are used.

This method was viewed in the seminar. Detailed information can be found [here](http://www.iro.umontreal.ca/~lisa/pointeurs/hierarchical-nnlm-aistats05.pdf) and practical using [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).    Briefly recall, for each word from the dictionary we define [Huffman code](https://en.wikipedia.org/wiki/Huffman_coding). More frequency words have shorter code. Then we construct the Huffman tree. In accordance with its code words are stored in leaves. In the non-leaf vertices are stored special hidden representations $h'$. 

$$\mathbb{P}[w_{t+i}|w_t] = \prod_{{h'}_{t+i}, r} \sigma\Big( \mathcal{r} \cdot {h'}^T_{t+i} v_t \Big) \text{, where}$$
* ${h'}_{t+i}$ – hidden vector of word's path $w_{t+i}$
* $\mathcal{r}$ – equals $-1$, if we turn left and $+1$ in another case
* $\sigma$ – sigmoid function

Using hierarchical softmax as approximation of softmax allows us to significantly reduce complexity of model training. 

#### Exercise
1. How can you implement skip-gram model as neural network.
2. Estimate complexity of one iteration of training the skip-gram model
  * $T$ - text size
  * $W$ - vocabulary size
  * $c$ - context radius
  * $d$ - embedding size
3. Estimate complexity of using hierarchical softmax in the skip-gram model.

#### Why sampling?
Let see at softmax funciton
$$\mathbb{P}[x_i] = \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)}$$
Optimizing this function is equivalent to optimizing its logarithm
$$\log \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)} = f(x_i|\theta) - log \sum_j \exp f(x_j|\theta)$$
Than gradient $\triangledown_{\theta} \mathbb{P}[x_i]$ is equal to
$$\triangledown_{\theta} f(x_i|\theta) - \sum_k \frac{\exp f(x_k|\theta)}{\sum_j \exp f(w_j|\theta)} \triangledown_{\theta} f(x_k|\theta)$$
Inside the sum we can find $\mathbb{P}[x_k]$ again. So, the previous expression can be written as
$$
\triangledown_{\theta} f(x_i|\theta) - \sum_k \mathbb{P}[x_k] \cdot \triangledown_{\theta} f(x_k|\theta) = 
\triangledown_{\theta} f(x_i|\theta) - \mathbb{E}_{x \sim \mathbb{P}[x]} \triangledown_{\theta} f(x|\theta)
$$
If we can sample $x \sim \mathbb{P}[x]$, then we has no sense to calculate the softmax itself. We just can train model, computing only gradient. It is possible due to the fact that we do not need the probability itself, but want to find representation of words only.

#### Negative Sampling
As the result, in practice, softmax loss functions is approximated with negative sampling, which looks like a series of logistic regressions
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t}
    \sum_{-c \leq i \leq c, i \neq 0} \Big(
        \mathbb{P}[y = 1 | w_{t+i}, w_t] +
        k \cdot \mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]} \mathbb{P}[y = 0 | \tilde{w}_{tj}, w_t]
    \Big)
$$
where $y=1$, if  word is a context of word and $y=0$ in another case. А $\tilde{w}_{tj}$ – sampling of words that don't belong to the context. Then main trick is how we will sample words $\mathbb{P}[y | w_{t+i}, w_t]$. In paper [noise contrastive estimation](https://www.cs.helsinki.fi/u/ahyvarin/papers/Gutmann10AISTATS.pdf) some interesting approaches is suggested. But we are going to use more simple variation
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t} \Big(
    \sum_{-c \leq i \leq c, i \neq 0} \big(
        \sigma({v'}_{t+i}^T v_t) +
        k \cdot \mathbb{E}_{w_j \sim \mathbb{Q}(w_j)} \sigma (-{v'}_{j}^T v_t)
    \big)
\Big),
$$
to find  $\mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]}$ we simply sample words $k$ times from distribution $\mathbb{Q}$, $\mathbb{P}(w_j|w_t) = \mathbb{Q}(w_j)$.

#### Additional word2vec tricks
For the best quality at the moment of learning it is proposed to use variety of tricks:
1. How to choose $\mathbb{Q}$.
2. Sampling of frequent and rare words.
3. The sampling of window size.
You are welcome to read about this in original papers.

#### Further reading
1. Briefly about both models [here](https://arxiv.org/pdf/1301.3781.pdf).
2. Hierarchical softmax and negative sampling [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

#### Exercises
1. Load pretrained word2vec embeddings.
2. Find top-10 nearest words again. Compare Euclidean and cosinus metric. Which is better? 
3. Find 10 nearest vector for expression v(king) - v(man) + v(women). If your model is tuned well, you find representation of word queen. 
4. How could you solve the following problems?
  * Find for country its capital
  * For a word find its plural form
  * For word define antonyms

In [ ]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
glove_vectors.most_similar('cat', topn=10)

[('dog', 0.6816747188568115),
 ('cats', 0.6815836429595947),
 ('pet', 0.5870364904403687),
 ('dogs', 0.540766716003418),
 ('feline', 0.48979705572128296),
 ('monkey', 0.48794347047805786),
 ('horse', 0.4732130467891693),
 ('pets', 0.4634858965873718),
 ('rabbit', 0.4608757495880127),
 ('leopard', 0.4585462808609009)]

In [ ]:
glove_vectors.most_similar_cosmul('cat')

[('dog', 0.8408365845680237),
 ('cats', 0.8407910466194153),
 ('pet', 0.7935174703598022),
 ('dogs', 0.7703826427459717),
 ('feline', 0.7448978424072266),
 ('monkey', 0.7439709901809692),
 ('horse', 0.7366058230400085),
 ('pets', 0.731742262840271),
 ('rabbit', 0.730437159538269),
 ('leopard', 0.7292724251747131)]

In [ ]:
glove_vectors.most_similar('loves', topn=10)

[('hates', 0.7345002889633179),
 ('likes', 0.6868317127227783),
 ('everybody', 0.6422836780548096),
 ('love', 0.6420263051986694),
 ('loved', 0.6281751394271851),
 ('knows', 0.6183099150657654),
 ('thinks', 0.6050291657447815),
 ('everyone', 0.5901667475700378),
 ('cares', 0.5732982158660889),
 ('dad', 0.5659530758857727)]

In [ ]:
glove_vectors.most_similar('clever', topn=10)

[('ingenious', 0.6507227420806885),
 ('deft', 0.6288391351699829),
 ('witty', 0.6275102496147156),
 ('inventive', 0.6236376762390137),
 ('amusing', 0.6079595685005188),
 ('smart', 0.5833670496940613),
 ('cunning', 0.5803178548812866),
 ('imaginative', 0.5728482007980347),
 ('crafty', 0.5691623687744141),
 ('shrewd', 0.5673002600669861)]

In [ ]:
glove_vectors.most_similar('mandarin', topn=10)

[('cantonese', 0.7189104557037354),
 ('putonghua', 0.5567854046821594),
 ('fluent', 0.554000735282898),
 ('hokkien', 0.5090120434761047),
 ('dialect', 0.4945122301578522),
 ('languages', 0.47307640314102173),
 ('language', 0.4720301330089569),
 ('dialects', 0.4664885401725769),
 ('arabic', 0.4631619155406952),
 ('creole', 0.4627479314804077)]

### 4. GloVe
This model combines two ideas:
1. Factorization of the matrix, as in LSA
2. Context of word is adjacent words in the corpus, just in word2vec.

Let matrix $X$ at position $i,j$ stores how many times the word $j$ was found in the context of word $i$. This matrix is filled with our corpus of text. Then the probability that the word $j$ appears in the context of word $i$ is equal to
$$P_{ij} = \frac{X_{ij}}{\sum_k X_{ik}}$$

At the same time we want to find some function $F$ and word embeddings that 
$$F((w_i - w_j)^T w_k) = \frac{F(w_i^T w_k)}{F(w_j^T w_k)} = \frac{P_{ik}}{P_{jk}}.$$
About motivation it's better to read in the original [paper](http://nlp.stanford.edu/pubs/glove.pdf).
Possible solution may be following
$$\exp(w_i^T w_j) = P_{ij}.$$

After a series of transformations you can receive that
$$w_i^T w_j = \log X_{ij} - \log\big(\sum_k X_{ik}\big)$$

Obviously, right part doesn't depend on $j$, but the resulting model is offered to be written symmetry as
$$w_i^T w_j + b_i + b_j = \log X_{ij}$$

As loss function authors suggest to use weighted MSE
$$\mathcal{L} = \sum_{i,j = 1}^{|W|} f(X_{ij}) \Big(w_i^T w_j + b_i + b_j - \log(X_{ij}) \Big)^2,$$
where $f$ is some weight function, defined for each pair of words. It is reasonable to store $X$ as sparse matrix. It is also noteworthy that during training only **not null** values of $X$ used. Model is trained with stochastic gradient descent. As some way of regularization the authors suggest to use two different matrices $W$ и $W'$ for word and context, so model can be rewritten as
$$w_i^T {w'}_j + b_i + b_j = \log X_{ij}$$
The resulted embedding of a word is the sum of two trained views $w + w'$.

#### Some additional GloVe tricks
1. In order to understand how you can choose $f$, please, read the original paper.
2. Also you are welcome to see, how $X$ is formed in the original article.

#### Further reading
1. GloVe [site](http://nlp.stanford.edu/projects/glove/).
2. Some words about [t-SNE](http://lvdmaaten.github.io/tsne/).

#### Exercises
1. Estimate complexity of model for one iteration. Choose appropriate $c$ for you.
2. Train GloVe word embeddings ($d$=256).
3. Check that v(king) - v(man) + v(women) is approximately equal v(queen).
4. Read about [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).
5. Use t-SNE to reduce the size of embeddings to 3. Make sure that the following groups of vectors are collinear (use visualization)
  * [man, woman], [Mr., Ms], [king, queen], etc
  * [CEO, company]
  * [adjective, its comparative form]

### 5. FastText
[FastText](https://arxiv.org/pdf/1607.04606.pdf) is a logical development of skip-gram model. The core feature is to present the word as the sum of its n-grams embeddings and its own embedding vector. For example, if we want to use only 3-grams, than word **fast** be reprented as n-grams **-fa**, **fas**, **ast**, **st-** and word **-fast-**. The context is encoeded in usual way, as a result similarity of word and context word is the following sum
$$s(w, c) = \sum_{n \in N_w} z^{T}_n v'_{c},$$
where $N_w$ is the set of all n-grams of word $w$ combined with word itself. The authors argue that the use of combination of 3-,4- and 5-grams helps to get better embeddings for rare and even unknown words (emdedding of word itself is null vector). For model training is proposed to use negative sampling.

You can find more information on [site](https://fasttext.cc) of FastText.

#### Exercises
1. Load FastText word embeddings.
2. Find some rare words in your corpus and find top-5 nearest words for them, using cosinus as metric.
3. Compare results with classic word2vec.
4. Invent some funny new word and found its top-5 nearest words again.
5. How could you compare all models. Suggest your metric (see papers for inspiration).